# Working with Vector Geodata - Quickstart

In this Tutorial we will read in some simple Geodata and play around with it!
First we need the library Geopandas which works very similiar to Pandas

In [ ]:
import geopandas as gpd
import folium
import rioxarray as riox
import numpy as np
import matplotlib.pyplot as plt
import os

There a many different data types for vector geodata like kml, shape, json, etc. We are working with geopackages which could be described as a normal dataframe, but with a geometry column. The geometry in a geodataframe can be Points, Lines and Polygons. 

For this quickstart example we want to look at the administrative boundaries of the city Bremen. For this city we also have all buildings.

Import a geodatafile (*.gpkg)

In [ ]:
#os.chdir("...")
neighborhoods_bremen = gpd.read_file("../data_example/Example_Bremen_Neighborhoods.gpkg")
buildings_bremen = gpd.read_file("../data_example/Example_Bremen_Buildings.gpkg")


Once we imported the files we can look at them in interctive maps via the geopandas built in function .explore() and the functions from folium.


In [ ]:
m = neighborhoods_bremen.explore(height=500, width=1000, name="Neighborhoods")
m = buildings_bremen.explore(m=m, color="red", name="Buildings")

folium.LayerControl().add_to(m)
m

So let´s say we are just interested in the neighborhood Westend and its buildings -
For that we slice the disctricts to just one element and clip (cut) the buildings to its geometry:

In [ ]:
westend = neighborhoods_bremen.loc[neighborhoods_bremen.Neighborhood_Name == "Westend"]
buildings_westend = buildings_bremen.clip(westend.geometry)

As you can see the newly created dataset just contains values for the neighborhood called Westend.

In [ ]:
m = westend.explore(height=500, width=1000, name="Neighborhoods")
m = buildings_westend.explore(m=m, color="red", name="Buildings")

folium.LayerControl().add_to(m)
m

# Vector Calculations - Building Density

With this type of data we could for example calculate the built-up density of the neighborhood. For this we need the area of the total neighborhood and the area of all buildings inside it. We wil use functions from geopandas. The function .area gets as the real area of the object in square meters.

In [ ]:
westend_area = westend.area.values[0]
buildings_area = np.sum(buildings_westend.area)
print(westend_area)
print(buildings_area)

In [ ]:
building_density = buildings_area / westend_area * 100
print(building_density)

Now we can see that in Westend about 28% of the whole neighborhood is covered with buildings.

# Vector Calculation II. Building Densities
Now it is an easy task to get the building densities for the whole neighborhood by just looping over it and iteratively calculate the building density neighborhood by neighborhood.

First we have to create an empty list where our building densities get stored. Then we loop over all neighborhood names and can slice our file into a smaller subset. For every subset (one neighborhood) we use the clip function to get just the relevant buildings. The sum of the building areas gets divided by the total area of the neighborhood - resulting the building density.

In [ ]:
building_density = []  # Empty List which gets filled iteratively
neighborhood_names = neighborhoods_bremen.Neighborhood_Name.values  # Needed for loop and indexing

for neighborhood in neighborhood_names:
    subset = neighborhoods_bremen.loc[neighborhoods_bremen.Neighborhood_Name == neighborhood]  # Get one neighborhood
    buildings_clip = buildings_bremen.clip(subset.geometry)  # Clip all buildings to one neighborhood
    building_density.append(np.sum(buildings_clip.area) / subset.area.values[0] * 100)  #  Building Area / Total Area = Density
    
neighborhoods_bremen["Building_Density"] = building_density  # Add new list to dataframe

If we want we can look at the first 5 rows of the new dataframe:

In [ ]:
neighborhoods_bremen.head()

# Visualization of the Results 
Now we want to visualize our Results again. Now we color our neighborhoods by building density. Blue is low - red is high.

In [ ]:
m = buildings_bremen.explore(color="gray", name="Buildings")
m = neighborhoods_bremen.explore(m=m, height=500, width=1000, name="Neighborhoods",
                             column = "Building_Density", scheme = "quantiles", cmap = "RdYlBu_r", legend = True)


folium.LayerControl().add_to(m)
m